### 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab10_1 Weight initialization
강의에서 다룬 Weight initialization 방법론들을 mnist data에 활용해보는 코드, 네트워크는 2개의 hidden layer를 가지는 (784, 512, 256, 10)의 구조


### Weight initialization : normal

In [1]:
import os, sys
import tensorflow as tf
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777) # for reproducibility
print(os.getcwd(),'\n', os.listdir())

D:\dev 
 ['.ipynb_checkpoints', 'Data structures and algorithms_warm_up.ipynb', 'Data structures and algorithms_week_1.ipynb', 'DLEL', 'DLEL.zip', 'DLEL_10_1_Weight_initialization.ipynb', 'logs', 'MNIST_data', 'py-automate', 'Untitled.ipynb', 'untitled.txt', 'Untitled1.ipynb']


In [2]:
## load data and parameter
mnist = input_data.read_data_sets('./MNIST_data/', one_hot = True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# graph setting
X = tf.placeholder(dtype = tf.float32, shape = [None, 784], name = 'Xinput')
Y = tf.placeholder(dtype = tf.float32, shape = [None, 10], name = 'Yinput')

with tf.variable_scope('layer1'):
    w1 = tf.Variable(initial_value = tf.random_normal(shape = [784, 512]), name = 'w1')
    b1 = tf.Variable(initial_value = tf.random_normal(shape = [512]), name = 'b1')
    layer1 = tf.sigmoid(tf.matmul(X, w1) + b1)
    layer1_hist = tf.summary.histogram(name = 'layer1', values = layer1)

with tf.variable_scope('layer2'):
    w2 = tf.Variable(initial_value = tf.random_normal(shape = [512, 256]), name = 'w2')
    b2 = tf.Variable(initial_value = tf.random_normal(shape = [256]), name = 'b2')
    layer2 = tf.sigmoid(tf.matmul(layer1, w2) + b2)
    layer2_hist = tf.summary.histogram(name = 'layer2', values = layer2)

with tf.variable_scope('layer3'):
    w3 = tf.Variable(initial_value = tf.random_normal(shape = [256, 10]), name = 'w3')
    b3 = tf.Variable(initial_value = tf.random_normal(shape = [10]), name = 'b3')
    score = tf.matmul(layer2, w3) + b3
    layer3 = tf.sigmoid(tf.matmul(layer2, w3) + b3)
    layer3_hist = tf.summary.histogram(name = 'layer3', values = layer3)

with tf.variable_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = Y))
    loss_scalar = tf.summary.scalar(name = 'loss', tensor = loss)

with tf.variable_scope('train'):
    train = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)
    
with tf.variable_scope('accuracy'):
    is_correct = tf.equal(tf.argmax(score, 1), tf.argmax(Y, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, dtype = tf.float32))
    accuracy_scalar = tf.summary.scalar(name = 'accuracy', tensor = accuracy)

In [4]:
# parameter 
batch_size = 100
epochs = 100

with tf.Session() as sess:
    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/normal')
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for epoch in range(epochs):
        
        for step in range(total_batch):
            
            tr_batch_xs, tr_batch_ys = mnist.train.next_batch(batch_size = batch_size)
            val_batch_xs, val_batch_ys = mnist.validation.next_batch(batch_size = mnist.validation.num_examples)
            summary, _, loss_tr = sess.run([merged_summary, train, loss], feed_dict = {X : tr_batch_xs, Y : tr_batch_ys})
            loss_val = sess.run(loss, feed_dict = {X : val_batch_xs, Y : val_batch_ys})
                
        if epoch % 5 == 0 or epoch == (epochs - 1):
            print('epoch : {:4}, tr_loss : {:.3f}, val_loss : {:.3f}'.format(epoch, loss_tr, loss_val))
        writer.add_summary(summary, global_step = epoch)
    
    print(sess.run(accuracy, feed_dict = {X : val_batch_xs, Y : val_batch_ys}))

epoch :    0, tr_loss : 3.565, val_loss : 3.189
epoch :    5, tr_loss : 1.171, val_loss : 1.224
epoch :   10, tr_loss : 1.070, val_loss : 0.932
epoch :   15, tr_loss : 0.550, val_loss : 0.793
epoch :   20, tr_loss : 0.718, val_loss : 0.708
epoch :   25, tr_loss : 0.617, val_loss : 0.648
epoch :   30, tr_loss : 0.388, val_loss : 0.605
epoch :   35, tr_loss : 0.717, val_loss : 0.572
epoch :   40, tr_loss : 0.562, val_loss : 0.546
epoch :   45, tr_loss : 0.675, val_loss : 0.523
epoch :   50, tr_loss : 0.332, val_loss : 0.505
epoch :   55, tr_loss : 0.360, val_loss : 0.489
epoch :   60, tr_loss : 0.236, val_loss : 0.476
epoch :   65, tr_loss : 0.419, val_loss : 0.464
epoch :   70, tr_loss : 0.234, val_loss : 0.453
epoch :   75, tr_loss : 0.333, val_loss : 0.443
epoch :   80, tr_loss : 0.142, val_loss : 0.436
epoch :   85, tr_loss : 0.389, val_loss : 0.428
epoch :   90, tr_loss : 0.216, val_loss : 0.420
epoch :   95, tr_loss : 0.305, val_loss : 0.414
epoch :   99, tr_loss : 0.177, val_loss 

### Weight initialization : Xavier initialization

In [5]:
tf.reset_default_graph()

# graph setting
X = tf.placeholder(dtype = tf.float32, shape = [None, 784], name = 'Xinput')
Y = tf.placeholder(dtype = tf.float32, shape = [None, 10], name = 'Yinput')

with tf.variable_scope('layer1'):
    w1 = tf.get_variable(name = 'w1', shape = [784, 512], initializer = tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(initial_value = tf.random_normal(shape = [512]), name = 'b1')
    layer1 = tf.sigmoid(tf.matmul(X, w1) + b1)
    layer1_hist = tf.summary.histogram(name = 'layer1', values = layer1)

with tf.variable_scope('layer2'):
    w2 = tf.get_variable(name = 'w2', shape = [512, 256], initializer = tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(initial_value = tf.random_normal(shape = [256]), name = 'b2')
    layer2 = tf.sigmoid(tf.matmul(layer1, w2) + b2)
    layer2_hist = tf.summary.histogram(name = 'layer2', values = layer2)

with tf.variable_scope('layer3'):
    w3 = tf.get_variable(name = 'w3', shape = [256, 10], initializer = tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(initial_value = tf.random_normal(shape = [10]), name = 'b3')
    score = tf.matmul(layer2, w3) + b3
    layer3 = tf.sigmoid(tf.matmul(layer2, w3) + b3)
    layer3_hist = tf.summary.histogram(name = 'layer3', values = layer3)

with tf.variable_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = Y))
    loss_scalar = tf.summary.scalar(name = 'loss', tensor = loss)

with tf.variable_scope('train'):
    train = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)
    
with tf.variable_scope('accuracy'):
    is_correct = tf.equal(tf.argmax(score, 1), tf.argmax(Y, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, dtype = tf.float32))
    accuracy_scalar = tf.summary.scalar(name = 'accuracy', tensor = accuracy)

In [6]:
# parameter 
batch_size = 100
epochs = 100

with tf.Session() as sess:
    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/xavier')
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for epoch in range(epochs):
        
        for step in range(total_batch):
            
            tr_batch_xs, tr_batch_ys = mnist.train.next_batch(batch_size = batch_size)
            val_batch_xs, val_batch_ys = mnist.validation.next_batch(batch_size = mnist.validation.num_examples)
            summary, _, loss_tr = sess.run([merged_summary, train, loss], feed_dict = {X : tr_batch_xs, Y : tr_batch_ys})
            loss_val = sess.run(loss, feed_dict = {X : val_batch_xs, Y : val_batch_ys})
                
        if epoch % 5 == 0 or epoch == (epochs - 1):
            print('epoch : {:4}, tr_loss : {:.3f}, val_loss : {:.3f}'.format(epoch, loss_tr, loss_val))
        writer.add_summary(summary, global_step = epoch)
    
    print(sess.run(accuracy, feed_dict = {X : val_batch_xs, Y : val_batch_ys}))

epoch :    0, tr_loss : 2.265, val_loss : 2.245
epoch :    5, tr_loss : 1.353, val_loss : 1.397
epoch :   10, tr_loss : 0.567, val_loss : 0.686
epoch :   15, tr_loss : 0.575, val_loss : 0.500
epoch :   20, tr_loss : 0.439, val_loss : 0.422
epoch :   25, tr_loss : 0.385, val_loss : 0.379
epoch :   30, tr_loss : 0.372, val_loss : 0.353
epoch :   35, tr_loss : 0.276, val_loss : 0.335
epoch :   40, tr_loss : 0.332, val_loss : 0.321
epoch :   45, tr_loss : 0.517, val_loss : 0.311
epoch :   50, tr_loss : 0.366, val_loss : 0.302
epoch :   55, tr_loss : 0.393, val_loss : 0.293
epoch :   60, tr_loss : 0.308, val_loss : 0.288
epoch :   65, tr_loss : 0.277, val_loss : 0.280
epoch :   70, tr_loss : 0.283, val_loss : 0.276
epoch :   75, tr_loss : 0.203, val_loss : 0.271
epoch :   80, tr_loss : 0.264, val_loss : 0.267
epoch :   85, tr_loss : 0.367, val_loss : 0.262
epoch :   90, tr_loss : 0.402, val_loss : 0.260
epoch :   95, tr_loss : 0.221, val_loss : 0.255
epoch :   99, tr_loss : 0.221, val_loss 

### Weight initialization : He initialization (with Relu activation)

In [7]:
# define He initializer
def he_init(n_input, n_output):
    stddev = tf.sqrt(2.0 / (n_input))
    return tf.truncated_normal_initializer(stddev = stddev)

In [8]:
tf.reset_default_graph()

# graph setting
X = tf.placeholder(dtype = tf.float32, shape = [None, 784], name = 'Xinput')
Y = tf.placeholder(dtype = tf.float32, shape = [None, 10], name = 'Yinput')

with tf.variable_scope('layer1'):
    w1 = tf.get_variable(name = 'w1', shape = [784, 512], initializer = he_init(784, 512))
    b1 = tf.Variable(initial_value = tf.random_normal(shape = [512]), name = 'b1')
    layer1 = tf.nn.relu(tf.matmul(X, w1) + b1)
    layer1_hist = tf.summary.histogram(name = 'layer1', values = layer1)

with tf.variable_scope('layer2'):
    w2 = tf.get_variable(name = 'w2', shape = [512, 256], initializer = he_init(512, 256))
    b2 = tf.Variable(initial_value = tf.random_normal(shape = [256]), name = 'b2')
    layer2 = tf.nn.relu(tf.matmul(layer1, w2) + b2)
    layer2_hist = tf.summary.histogram(name = 'layer2', values = layer2)

with tf.variable_scope('layer3'):
    w3 = tf.get_variable(name = 'w3', shape = [256, 10], initializer = he_init(256, 10))
    b3 = tf.Variable(initial_value = tf.random_normal(shape = [10]), name = 'b3')
    score = tf.matmul(layer2, w3) + b3
    layer3 = tf.nn.relu(tf.matmul(layer2, w3) + b3)
    layer3_hist = tf.summary.histogram(name = 'layer3', values = layer3)

with tf.variable_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = Y))
    loss_scalar = tf.summary.scalar(name = 'loss', tensor = loss)

with tf.variable_scope('train'):
    train = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)
    
with tf.variable_scope('accuracy'):
    is_correct = tf.equal(tf.argmax(score, 1), tf.argmax(Y, axis = 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, dtype = tf.float32))
    accuracy_scalar = tf.summary.scalar(name = 'accuracy', tensor = accuracy)

In [9]:
# parameter 
batch_size = 100
epochs = 100

with tf.Session() as sess:
    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs/he')
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for epoch in range(epochs):
        
        for step in range(total_batch):
            
            tr_batch_xs, tr_batch_ys = mnist.train.next_batch(batch_size = batch_size)
            val_batch_xs, val_batch_ys = mnist.validation.next_batch(batch_size = mnist.validation.num_examples)
            summary, _, loss_tr = sess.run([merged_summary, train, loss], feed_dict = {X : tr_batch_xs, Y : tr_batch_ys})
            loss_val = sess.run(loss, feed_dict = {X : val_batch_xs, Y : val_batch_ys})
                
        if epoch % 5 == 0 or epoch == (epochs - 1):
            print('epoch : {:4}, tr_loss : {:.3f}, val_loss : {:.3f}'.format(epoch, loss_tr, loss_val))
        writer.add_summary(summary, global_step = epoch)
    
    print(sess.run(accuracy, feed_dict = {X : val_batch_xs, Y : val_batch_ys}))

epoch :    0, tr_loss : 0.397, val_loss : 0.490
epoch :    5, tr_loss : 0.284, val_loss : 0.266
epoch :   10, tr_loss : 0.268, val_loss : 0.217
epoch :   15, tr_loss : 0.141, val_loss : 0.182
epoch :   20, tr_loss : 0.136, val_loss : 0.154
epoch :   25, tr_loss : 0.098, val_loss : 0.133
epoch :   30, tr_loss : 0.086, val_loss : 0.119
epoch :   35, tr_loss : 0.091, val_loss : 0.110
epoch :   40, tr_loss : 0.030, val_loss : 0.101
epoch :   45, tr_loss : 0.053, val_loss : 0.094
epoch :   50, tr_loss : 0.103, val_loss : 0.089
epoch :   55, tr_loss : 0.086, val_loss : 0.085
epoch :   60, tr_loss : 0.068, val_loss : 0.082
epoch :   65, tr_loss : 0.090, val_loss : 0.079
epoch :   70, tr_loss : 0.023, val_loss : 0.078
epoch :   75, tr_loss : 0.142, val_loss : 0.075
epoch :   80, tr_loss : 0.030, val_loss : 0.075
epoch :   85, tr_loss : 0.035, val_loss : 0.073
epoch :   90, tr_loss : 0.021, val_loss : 0.073
epoch :   95, tr_loss : 0.025, val_loss : 0.071
epoch :   99, tr_loss : 0.014, val_loss 

### Tensorboard : graph
![Alt text](http://i.imgur.com/ySlvADi.png)

### Tensorboard : accuracy & loss  
주황색은 Relu activation function을 사용하고 He initialization를 사용한 경우로 학습이 가장 빨리 이루어짐을 확인이 가능하다. sigmoid activation function을 쓴 하늘색과 보라색의 경우 xavier initialization을 사용한 경우가 학습이 빨리 이루어진 것을 확인할 수 있다.
![Alt text](http://i.imgur.com/U6rw0uy.png)

### Tensorboard : accuracy & loss  
주황색은 Relu activation을 쓰고 He initialization을 쓴 경우로 다른 두 경우와 직접적인 비교가 어렵다. 나머지 sigmoid activation function을 쓴 경우, xavier initialization을 쓴 경우 activation 값이 정규분포모양의 형태로 고루퍼져있어 neural net의 표현력이 좋고, vanishing gradient 문제를 해결할 수 있음을 확인할 수 있다.
![Alt text](http://i.imgur.com/hbZTCy2.png)